In [1]:
from importlib.util import find_spec
%load_ext autoreload
%autoreload 2

%matplotlib inline

if find_spec("qml_hep_lhc") is None:
    import sys
    sys.path.append('../..')

In [3]:
from qml_hep_lhc.data import ElectronPhoton, MNIST, QuarkGluon
from qml_hep_lhc.models import QCNNSandwich
from tensorflow.keras.callbacks import ReduceLROnPlateau
import argparse
import numpy as np

import matplotlib.pyplot as plt
import time

In [17]:
args = argparse.Namespace()

# Data
args.center_crop = 0.7
args.resize = [8,8]
args.standardize = 1
args.binary_data = [0,1]
args.batch_size = 64
args.validation_split = 0.05
args.labels_to_categorical = 1

# Base Model
args.learning_rate = 1e-3
args.epochs = 10

# Quantum CNN Parameters
args.n_layers = 1
args.ansatz_class = 'NQubit'
args.sparse = 1
args.num_conv_layers = 2
args.conv_dims = [1,1]
args.num_qconv_layers = 2
args.qconv_dims = [1,1]

In [12]:
data = MNIST(args)
data.prepare_data()
data.setup()
print(data)

Binarizing data...
Binarizing data...
Center cropping...
Center cropping...
Resizing data...
Resizing data...
Standardizing data...
Converting labels to categorical...
Converting labels to categorical...

Dataset :MNIST
╒════════╤══════════════════╤════════════════╤═════════════════╤═══════════╕
│ Data   │ Train size       │ Val size       │ Test size       │ Dims      │
╞════════╪══════════════════╪════════════════╪═════════════════╪═══════════╡
│ X      │ (12031, 8, 8, 1) │ (634, 8, 8, 1) │ (2115, 8, 8, 1) │ (8, 8, 1) │
├────────┼──────────────────┼────────────────┼─────────────────┼───────────┤
│ y      │ (12031, 2)       │ (634, 2)       │ (2115, 2)       │ (2,)      │
╘════════╧══════════════════╧════════════════╧═════════════════╧═══════════╛

╒══════════════╤═══════╤═══════╤════════╤═══════╤══════════════════════════╕
│ Type         │   Min │   Max │   Mean │   Std │ Samples for each class   │
╞══════════════╪═══════╪═══════╪════════╪═══════╪══════════════════════════╡
│ Train I

In [18]:
data.config()

{'input_dims': (8, 8, 1), 'output_dims': (2,), 'mapping': [0, 1]}

In [19]:
model = QCNNSandwich(data.config(), args)

../../qml_hep_lhc/layers/qconv2d.py:62: UserWarning: n_qubits is only used for NQubit ansatz.
  warnings.warn('n_qubits is only used for NQubit ansatz.')


In [20]:
model.compile()
model.build_graph().summary()

Model: "QCNNSandwich-AngleMap-Chen"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 8, 8, 1)]         0         
                                                                 
 conv2d_5 (Conv2D)           (None, 8, 8, 1)           10        
                                                                 
 conv2d_6 (Conv2D)           (None, 8, 8, 1)           10        
                                                                 
 batch_normalization_1 (Batc  (None, 8, 8, 1)          4         
 hNormalization)                                                 
                                                                 
 qconv2d_0 (QConv2D)         (None, 8, 8, 1, 1)        45        
                                                                 
 qconv2d_1 (QConv2D)         (None, 8, 8, 1, 1)        45        
                                        

In [21]:
lr_scheduler_callback = ReduceLROnPlateau(monitor='val_loss',
                                              factor=np.sqrt(0.1),
                                              patience=5,
                                              min_delta=0.0001,
                                              min_lr=1e-8)
callbacks = [lr_scheduler_callback]
model.fit(data, callbacks)

NameError: name 'callbacks' is not defined

In [ ]:
model.test(data, callbacks=callbacks)